# Resnet18 应用于 CIFAR10 分类

终于可以说一下Resnet分类网络了，它差不多是当前应用最为广泛的CNN特征提取网络。细节大家可以参考这篇论文：

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun, Deep Residual Learning for Image Recognition, CVPR 2016.

我们的一般印象当中，CNN越深就越具有更强的表达能力。因此，CNN分类网络自Alexnet的7层发展到了VGG的19层，后来有了Googlenet的22层。可后来我们发现深度CNN网络达到一定深度后再一味地增加层数并不能带来进一步地分类性能提高，反而会招致网络收敛变得更慢，分类准确率也变得更差。

## 1. BasicBlock

何恺明想到了常规视觉领域常用的residual representation的概念，并进一步将它应用在了CNN模型的构建当中，于是就有了基本的residual learning的 BasicBlock。


BasicBlock 通过Identity mapping的引入在输入、输出之间建立了一条直接的关联通道，从而使得强大的有参层集中精力学习输入、输出之间的残差。简来说，残差更小，学习起来更容易，也收敛更快。

下面我们看一下 BasicBlock 的代码：

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim



class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

代码比较简单，非常容易看懂，需要注意的是，stride 步长可能是1，也可能是2。如果步长是2的话，两个 feature map 尺寸不一样，就不能做加法了；同样，如果两个 feature map 的通道数不一样，也无法做加法。所有，有一个判断专门处理这个。

## 2. Bottleneck Block

为了实际计算的考虑，作者提出了一种bottleneck的结构块来代替上面的 Residual block，它像Inception网络那样通过使用1x1 conv来巧妙地缩减或扩张feature map维度从而使得 3x3 的卷积核数目不受外界即上一层输入的影响，自然它的输出也不会影响到下一层，具体图示如下：


实现的代码如下所示：


In [2]:
class Bottleneck(nn.Module):
    expansion = 4
    
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

## 3. 创建dataloader

In [3]:
# 使用GPU训练，可以在菜单 "代码执行工具" -> "更改运行时类型" 里进行设置
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 4. 创建 ResNet18 网络

In [4]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.num_classes = 10
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(BasicBlock, 64,  2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        self.linear = nn.Linear(512, self.num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  
        # 64 * 32 * 32
        out = self.layer1(out)
        # 64 * 32 * 32
        out = self.layer2(out)
        # 128 * 16 * 16    
        out = self.layer3(out)
        # 256 * 8 * 8   
        out = self.layer4(out)
        # 512 * 4 * 4        
        out = F.avg_pool2d(out, 4)
        # 512 * 1 * 1
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

实例化网络：

In [5]:
# 网络放到GPU上
net = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

## 5. 模型训练：

训练代码和之前的完全一样。

In [6]:
for epoch in range(10):  # 重复多轮训练
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 输出统计信息
        if i % 100 == 0:   
            print('Epoch: %d Minibatch: %5d loss: %.3f' %(epoch + 1, i + 1, loss.item()))

print('Finished Training')

Epoch: 1 Minibatch:     1 loss: 2.384
Epoch: 1 Minibatch:   101 loss: 1.613
Epoch: 1 Minibatch:   201 loss: 1.326
Epoch: 1 Minibatch:   301 loss: 1.268
Epoch: 2 Minibatch:     1 loss: 1.031
Epoch: 2 Minibatch:   101 loss: 1.089
Epoch: 2 Minibatch:   201 loss: 1.009
Epoch: 2 Minibatch:   301 loss: 0.759
Epoch: 3 Minibatch:     1 loss: 0.796
Epoch: 3 Minibatch:   101 loss: 0.918
Epoch: 3 Minibatch:   201 loss: 1.030
Epoch: 3 Minibatch:   301 loss: 0.682
Epoch: 4 Minibatch:     1 loss: 0.602
Epoch: 4 Minibatch:   101 loss: 0.847
Epoch: 4 Minibatch:   201 loss: 0.736
Epoch: 4 Minibatch:   301 loss: 0.710
Epoch: 5 Minibatch:     1 loss: 0.502
Epoch: 5 Minibatch:   101 loss: 0.570
Epoch: 5 Minibatch:   201 loss: 0.600
Epoch: 5 Minibatch:   301 loss: 0.580
Epoch: 6 Minibatch:     1 loss: 0.461
Epoch: 6 Minibatch:   101 loss: 0.485
Epoch: 6 Minibatch:   201 loss: 0.503
Epoch: 6 Minibatch:   301 loss: 0.592
Epoch: 7 Minibatch:     1 loss: 0.366
Epoch: 7 Minibatch:   101 loss: 0.437
Epoch: 7 Min

## 6. 模型测试

In [7]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 85.33 %


一个神奇的时刻，准确率在 InceptionV3 的基础上又提高了 0.56 个百分点 ~

其实我是故意的，提升是非常容易的，多跑几个 epoch 就行了。

现在还留有余地  因为有了残差学习，我们非常容易的就能够把网络加深，到50层，100层 

精力有限，这里我就不再跑 ResNet50 了，但代码提供在下面，感兴趣的同学可以自己跑一跑。

In [ ]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(Bottleneck, 64,  3, stride=1)
        self.layer2 = self._make_layer(Bottleneck, 128, 4, stride=2)
        self.layer3 = self._make_layer(Bottleneck, 256, 6, stride=2)
        self.layer4 = self._make_layer(Bottleneck, 512, 3, stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out